In [28]:
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter

from pytket.extensions.qiskit import AerBackend
from pytket.backends.backendresult import BackendResult
import matplotlib.pyplot as plt
import numpy as np

from pytket.circuit import QControlBox, CircBox, StatePreparationBox
from pytket.backends.backendresult import BackendResult
from pytket.utils import QubitPauliOperator, gen_term_sequence_circuit
from pytket import Qubit, Circuit
from pytket.passes import DecomposeBoxes
from pytket.circuit.display import render_circuit_jupyter
from pytket.pauli import Pauli, QubitPauliString
from pytket.extensions.nexus import NexusBackend, QuantinuumConfig, Nexus

In [29]:
def build_ising_approximation() -> Circuit:
    # t is pi/2
    # 1 Trotter step
    
    zz1 = QubitPauliString({Qubit(0):Pauli.Z, Qubit(1):Pauli.Z})
    zz2 = QubitPauliString({Qubit(1):Pauli.Z, Qubit(2):Pauli.Z})
    zz3 = QubitPauliString({Qubit(0):Pauli.Z, Qubit(2):Pauli.Z})

    H = QubitPauliOperator({zz1: -1, zz2: -1, zz3: -1})

    circ = gen_term_sequence_circuit(H, Circuit(3))
        
    return circ

In [30]:
def build_qft_circuit(n_qubits: int) -> Circuit:
    circ = Circuit(n_qubits, name="QFT")
    for i in range(n_qubits):
        circ.H(i)
        for j in range(i + 1, n_qubits):
            circ.CU1(1 / 2 ** (j - i), j, i)
    for k in range(0, n_qubits // 2):
        circ.SWAP(k, n_qubits - k - 1)
    return circ

In [31]:
def build_phase_estimation_circuit(
    n_measurement_qubits: int, state_prep_circuit: Circuit, unitary_circuit: Circuit
) -> Circuit:
    # Define a Circuit with a measurement and prep register
    qpe_circ: Circuit = Circuit()
    n_state_prep_qubits = state_prep_circuit.n_qubits
    measurement_register = qpe_circ.add_q_register("m", n_measurement_qubits)
    state_prep_register = qpe_circ.add_q_register("p", n_state_prep_qubits)
    qpe_circ.add_circuit(state_prep_circuit, list(state_prep_register))

    # Create a controlled unitary with a single control qubit
    unitary_circuit.name = "U"
    controlled_u_gate = QControlBox(CircBox(unitary_circuit), 1)

    # Add Hadamard gates to every qubit in the measurement register
    for m_qubit in measurement_register:
        qpe_circ.H(m_qubit)

    # Add all (2**n_measurement_qubits - 1) of the controlled unitaries sequentially
    for m_qubit in range(n_measurement_qubits):
        control_index = n_measurement_qubits - m_qubit - 1
        control_qubit = [measurement_register[control_index]]
        for _ in range(2**m_qubit):
            qpe_circ.add_qcontrolbox(
                controlled_u_gate, control_qubit + list(state_prep_register)
            )

    # Finally, append the inverse qft and measure the qubits
    qft_box = CircBox(build_qft_circuit(n_measurement_qubits))
    inverse_qft_box = qft_box.dagger
    qpe_circ.add_circbox(inverse_qft_box, list(measurement_register))
    qpe_circ.measure_register(measurement_register, "c")
    return qpe_circ

In [32]:
from datetime import datetime

# circuit to prepare the eigenstate of H for the specified state
prep_circuit = Circuit(3)

# Base unitary for controlled U ops
unitary_circuit = build_ising_approximation()
for i in range(1,7):
    std_phase_est = Nexus().new_project(f"Std Phase Estimation - Ising - prec: {i}, {datetime.now()}")

    configuration = QuantinuumConfig(device_name="H1-1E")
    quantinuum_backend = NexusBackend(
            backend_config= configuration, 
            project= std_phase_est
        )
    
    qpe_circ = build_phase_estimation_circuit(i, prep_circuit, unitary_circuit)
    backend = quantinuum_backend
    DecomposeBoxes().apply(qpe_circ)
    compiled_circ = backend.get_compiled_circuit(qpe_circ)
    print('num gates:', compiled_circ.n_gates)
    print('depth:', compiled_circ.depth())
    n_shots = 100
    result = backend.run_circuit(compiled_circ, n_shots)
    sorted_shots = result.get_counts().most_common()
    print(sorted_shots)


New project created: Std Phase Estimation - Ising - prec: 1, 2024-02-04 12:58:41.943760

Started using project with name: Std Phase Estimation - Ising - prec: 1, 2024-02-04 12:58:41.943760
num gates: 26
depth: 21
[((0,), 51), ((1,), 49)]

New project created: Std Phase Estimation - Ising - prec: 2, 2024-02-04 12:59:12.743951

Started using project with name: Std Phase Estimation - Ising - prec: 2, 2024-02-04 12:59:12.743951
num gates: 70
depth: 60
[((1, 1), 94), ((0, 0), 3), ((1, 0), 2), ((0, 1), 1)]

New project created: Std Phase Estimation - Ising - prec: 3, 2024-02-04 13:00:13.672114

Started using project with name: Std Phase Estimation - Ising - prec: 3, 2024-02-04 13:00:13.672114
num gates: 155
depth: 138
[((1, 1, 0), 79), ((0, 1, 0), 10), ((0, 0, 0), 4), ((1, 1, 1), 4), ((1, 0, 0), 3)]

New project created: Std Phase Estimation - Ising - prec: 4, 2024-02-04 13:01:39.660827

Started using project with name: Std Phase Estimation - Ising - prec: 4, 2024-02-04 13:01:39.660827
num 

In [9]:
def plot_qpe_results(
    sim_result: BackendResult,
    n_strings: int = 4,
    dark_mode: bool = False,
    y_limit: int = 1000,
) -> None:
    """
    Plots results in a barchart given a BackendResult. the number of stings displayed
    can be specified with the n_strings argument.
    """
    counts_dict = sim_result.get_counts()
    sorted_shots = counts_dict.most_common()
    n_most_common_strings = sorted_shots[:n_strings]
    x_axis_values = [str(entry[0]) for entry in n_most_common_strings]  # basis states
    y_axis_values = [entry[1] for entry in n_most_common_strings]  # counts
    if dark_mode:
        plt.style.use("dark_background")
    fig = plt.figure()
    ax = fig.add_axes((0, 0, 0.75, 0.5))
    color_list = ["orange"] * (len(x_axis_values))
    ax.bar(
        x=x_axis_values,
        height=y_axis_values,
        color=color_list,
    )
    ax.set_title(label="Results")
    plt.ylim([0, y_limit])
    plt.xlabel("Basis State")
    plt.ylabel("Number of Shots")
    plt.show()

In [10]:
plot_qpe_results(result, y_limit=int(1.2 * n_shots))

NameError: name 'result' is not defined

In [ ]:
def single_phase_from_backendresult(result: BackendResult) -> float:
    # Extract most common measurement outcome
    basis_state = result.get_counts().most_common()[0][0]
    bitstring = "".join([str(bit) for bit in basis_state])
    integer_j = int(bitstring, 2)

    # Calculate theta estimate
    return integer_j / (2 ** len(bitstring))

def average_phase_from_backendresult(result: BackendResult) -> float:
    top4 = result.get_counts().most_common()[:4]
    print(top4)
    sum = top4[0][1]+ top4[1][1] + top4[2][1] + top4[3][1]
    bitstrings = [ "".join([str(bit) for bit in basis_state[0]]) for basis_state in top4]
    integers = [ int(bitstring, 2) for bitstring in bitstrings]
    avg = 0
    for i in range(4):
        avg = avg + top4[i][1]/sum * integers[i]
    return avg / (2 ** len(bitstrings[0]))
